# CIFAR-10 Image Classification Challenge

**Multi-class image classification**

Your challenge is to develop a machine learning model for classifying images in the classic [CIFAR-10 dataset](https://www.cs.toronto.edu/~kriz/cifar.html).

This is a great opportunity to learn about some core concepts and ideas in **deep learning** and **computer vision** using the popular machine learning library, [PyTorch](https://pytorch.org/). In particular, this challenge will give you the chance to learn about training and using convolutional neural networks (CNNs), which are ubiquitous in deep learning.

This Jupyter notebook will guide you through some of the various general stages involved in machine learning projects, including data visualisation, data preprocessing, model training and model evaluation, and afterwards, you will be able to submit your test set predictions for evaluation on the [DOXA AI](https://doxaai.com/competition/cifar-10) platform.

**Before you get started, make sure to [sign up for an account](https://doxaai.com/sign-up) if you do not already have one and [enrol to take part](https://doxaai.com/competition/cifar-10) in the challenge.**

**If you have any questions, feel free to ask them in the [DOXA Community Discord server](https://discord.gg/MUvbQ3UYcf).**


## Background

The [CIFAR-10 dataset](https://www.cs.toronto.edu/~kriz/cifar.html) is a classic image dataset in machine learning collected by Alex Krizhevsky, Vinod Nair, and Geoffrey Hinton. Its training set consists of 50,000 32&times;32 colour images, where each image belongs to one of ten classes: `airplane`, `automobile`, `bird`, `cat`, `deer`, `dog`, `frog`, `horse`, `ship` and `truck`.

![CIFAR-10](https://pytorch.org/tutorials/_images/cifar10.png)


## Installing and importing useful packages

To get started, we will install and import a few packages we will need. To install PyTorch (including `torchvision`) if you do not already have it, check out the PyTorch [installation instructions](https://pytorch.org/get-started/locally/).


In [ ]:
# %pip install numpy matplotlib torchinfo
%pip install -U doxa-cli

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Loading & exploring the data

First, we'll download the data if we don't already have it.


In [ ]:
import os

# Download the dataset if we do not already have it
if not os.path.exists("data"):
    os.makedirs("data")

    !curl -L https://github.com/DoxaAI/educational-challenges/releases/download/cifar-10/cifar-10-data.npz --output data/data.npz

# Load the data
data = np.load("data/data.npz")

data

The `data.npz` file is a compressed NumPy zip archive containing the training dataset features (i.e. the images), the training dataset targets (the ten class labels encoded ordinally) and the test dataset features.

Each image in our 50,000-image training dataset has the shape `(3, 32, 32)`, which corresponds to the number of channels (red, green and blue), the height and the width of each image. Each value is encoded as an 8-bit unsigned integer, where values range from `0` to `255`.

Let's quickly verify this by checking the shapes of these three NumPy arrays and inspecting the raw pixel values of the first image in the training dataset:


In [ ]:
print("Training features shape:", data["training_data"].shape)
print("Training targets shape: ", data["training_targets"].shape)
print("Test features shape:    ", data["test_data"].shape)

data["training_data"][0]

Now, let's visualise the data by looking at a subset of the images in the training dataset:


In [ ]:
grid = torchvision.utils.make_grid(torch.from_numpy(data["training_data"][:16]))

# imshow() expects images to be (height, width, channels) rather than (channels, height, width)
plt.imshow(grid.permute(1, 2, 0))

We can also take a look at what these images are supposed to be:


In [ ]:
CLASSES = (
    "plane",
    "car",
    "bird",
    "cat",
    "deer",
    "dog",
    "frog",
    "horse",
    "ship",
    "truck",
)

[CLASSES[c] for c in data["training_targets"][:16]]

## Preparing the data

First, let's split the training data we have been given into a training set and a separate validation set on which we can test the models we train.


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(
    data["training_data"], data["training_targets"], test_size=0.2
)

X_train.shape, y_train.shape, X_valid.shape, y_valid.shape

Now, let's create a PyTorch [`VisionDataset`](https://pytorch.org/vision/main/generated/torchvision.datasets.VisionDataset.html) implementation so that we can work with the data more easily.


In [ ]:
from typing import Any, Callable


class CIFAR10(torchvision.datasets.VisionDataset):
    def __init__(
        self,
        transform: Callable[..., Any],
        features: np.ndarray,
        targets: np.ndarray = None,
    ) -> None:
        super().__init__(transform=transform)

        self.features = torch.from_numpy(features)
        self.targets = torch.from_numpy(targets) if targets is not None else None

    def __getitem__(self, index: int) -> Any:
        if self.targets is not None:
            return self.transform(self.features[index]), self.targets[index]

        return self.transform(self.features[index])

    def __len__(self) -> int:
        return self.features.shape[0]

We saw earlier how image pixel values are currently encoded as unsigned integers in the range `[0, 255]`. Before we can use these images in training, we need to rescale the images to be something that is more neural network friendly (e.g. the range `[0, 1]` or `[-1, 1]`). In this example, we will just rescale the image pixel values to be in the range `[-1, 1]`, since standardising (or normalising) the data fed into neural networks can aid with convergence during training.

[Transforms](https://pytorch.org/vision/main/transforms.html) in the `torchvision` library are a convenient way of applying common image transformations in computer vision, such as scaling, resizing, cropping, flipping, rotating and more. When you are working with limited training data, you may wish to **augment the training dataset**, such as by training off images that have been randomly cropped, flipped, rotated, padded or rescaled, in order to improve performance. The transforms below are re-applied every epoch, so adding, say, [`RandomResizedCrop`](https://pytorch.org/vision/main/generated/torchvision.transforms.v2.RandomResizedCrop.html#torchvision.transforms.v2.RandomResizedCrop) to the list of training transforms below will potentially transform each image differently each time. You should only augment the training dataset (not the test dataset!).

Check out the PyTorch documentation for [transforming and augmenting images](https://pytorch.org/vision/main/transforms.html#v2-api-reference-recommended) to find out more. You may wish to experiment with different transforms to see how you can improve the performance of your model!


In [ ]:
from torchvision.transforms import v2

training_transform = v2.Compose(
    [
        # EXERCISE: what other transforms could you perform and what order should you do them in?
        v2.ToDtype(torch.float32, scale=True),
        v2.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
    ]
)

# We will use this on both our validation and test sets
test_transform = v2.Compose(
    [
        v2.ToDtype(torch.float32, scale=True),
        v2.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
    ]
)

In [ ]:
training_dataset = CIFAR10(
    features=X_train,
    targets=y_train,
    transform=training_transform,
)

validation_dataset = CIFAR10(
    features=X_valid,
    targets=y_valid,
    transform=test_transform,
)

## Defining a model

We are now ready to define the structure of our convolutional neural network!

It is very simple: it has two convolutional layers (each followed by a `ReLU` and a max-pooling operation) and then two fully connected linear layers. The final linear layer (named `linear2`) is not wrapped with an activation function since the PyTorch [`CrossEntropyLoss`](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html) loss function we will use in training expects its inputs to be the _unnormalised_ logits for each class. If you are using a different deep learning framework, make sure to check this, as it could be different.

If you are curious, check out the [different layers](https://pytorch.org/docs/main/nn.html) available in the PyTorch documentation.


In [ ]:
import torch.nn as nn


#########################################
#       Improve this basic model!       #
#########################################


class Model(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=8, kernel_size=3)
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.linear1 = nn.Linear(in_features=576, out_features=64)
        self.linear2 = nn.Linear(in_features=64, out_features=10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = torch.flatten(x, 1)

        x = torch.relu(self.linear1(x))
        x = self.linear2(x)

        return x


# Instantiate the model and move it onto the GPU (if one is available)
model = Model().to(device)

model

The [`torchinfo`](https://github.com/TylerYep/torchinfo) package provides a useful `summary()` function that can provide some additional information about our model, such as input and output tensor shapes, as well as the number of trainable parameters. If your model has a huge number of trainable parameters (often from a large `Linear` layer), it will require more memory (and may potentially be harder to train in certain circumstances).


In [ ]:
from torchinfo import summary

summary(
    model,
    input_size=(1, 3, 32, 32),
    col_names=["input_size", "output_size", "num_params", "trainable"],
)

Using this [great online tool](https://alexlenail.me/NN-SVG/LeNet.html) by Alexander LeNail, we can visualise the architecture of this neural network:

<svg width="1071.9" height="332.8" version="1.1" xmlns="http://www.w3.org/2000/svg">
 <g transform="translate(-786.66 -402.43)">
  <g transform="matrix(1.7411 0 0 1.7411 -609.7 -397.81)">
   <g stroke="#000">
    <rect class="rect" x="802.5" y="527" width="32" height="32" fill="#a0a0a0" opacity=".8"/>
    <rect class="rect" x="809.5" y="534" width="32" height="32" fill="#e0e0e0" opacity=".8"/>
    <rect class="rect" x="816.5" y="541" width="32" height="32" fill="#a0a0a0" opacity=".8"/>
    <rect class="rect" x="888.5" y="510.5" width="30" height="30" fill="#a0a0a0" opacity=".8"/>
    <rect class="rect" x="895.5" y="517.5" width="30" height="30" fill="#e0e0e0" opacity=".8"/>
    <rect class="rect" x="902.5" y="524.5" width="30" height="30" fill="#a0a0a0" opacity=".8"/>
    <rect class="rect" x="909.5" y="531.5" width="30" height="30" fill="#e0e0e0" opacity=".8"/>
    <rect class="rect" x="916.5" y="538.5" width="30" height="30" fill="#a0a0a0" opacity=".8"/>
    <rect class="rect" x="923.5" y="545.5" width="30" height="30" fill="#e0e0e0" opacity=".8"/>
    <rect class="rect" x="930.5" y="552.5" width="30" height="30" fill="#a0a0a0" opacity=".8"/>
    <rect class="rect" x="937.5" y="559.5" width="30" height="30" fill="#e0e0e0" opacity=".8"/>
    <rect class="rect" x="977.5" y="518" width="15" height="15" fill="#a0a0a0" opacity=".8"/>
    <rect class="rect" x="984.5" y="525" width="15" height="15" fill="#e0e0e0" opacity=".8"/>
    <rect class="rect" x="991.5" y="532" width="15" height="15" fill="#a0a0a0" opacity=".8"/>
    <rect class="rect" x="998.5" y="539" width="15" height="15" fill="#e0e0e0" opacity=".8"/>
    <rect class="rect" x="1005.5" y="546" width="15" height="15" fill="#a0a0a0" opacity=".8"/>
    <rect class="rect" x="1012.5" y="553" width="15" height="15" fill="#e0e0e0" opacity=".8"/>
    <rect class="rect" x="1019.5" y="560" width="15" height="15" fill="#a0a0a0" opacity=".8"/>
    <rect class="rect" x="1026.5" y="567" width="15" height="15" fill="#e0e0e0" opacity=".8"/>
    <rect class="rect" x="1053.5" y="491" width="13" height="13" fill="#a0a0a0" opacity=".8"/>
    <rect class="rect" x="1060.5" y="498" width="13" height="13" fill="#e0e0e0" opacity=".8"/>
    <rect class="rect" x="1067.5" y="505" width="13" height="13" fill="#a0a0a0" opacity=".8"/>
    <rect class="rect" x="1074.5" y="512" width="13" height="13" fill="#e0e0e0" opacity=".8"/>
    <rect class="rect" x="1081.5" y="519" width="13" height="13" fill="#a0a0a0" opacity=".8"/>
    <rect class="rect" x="1088.5" y="526" width="13" height="13" fill="#e0e0e0" opacity=".8"/>
    <rect class="rect" x="1095.5" y="533" width="13" height="13" fill="#a0a0a0" opacity=".8"/>
    <rect class="rect" x="1102.5" y="540" width="13" height="13" fill="#e0e0e0" opacity=".8"/>
    <rect class="rect" x="1109.5" y="547" width="13" height="13" fill="#a0a0a0" opacity=".8"/>
    <rect class="rect" x="1116.5" y="554" width="13" height="13" fill="#e0e0e0" opacity=".8"/>
    <rect class="rect" x="1123.5" y="561" width="13" height="13" fill="#a0a0a0" opacity=".8"/>
    <rect class="rect" x="1130.5" y="568" width="13" height="13" fill="#e0e0e0" opacity=".8"/>
    <rect class="rect" x="1137.5" y="575" width="13" height="13" fill="#a0a0a0" opacity=".8"/>
    <rect class="rect" x="1144.5" y="582" width="13" height="13" fill="#e0e0e0" opacity=".8"/>
    <rect class="rect" x="1151.5" y="589" width="13" height="13" fill="#a0a0a0" opacity=".8"/>
    <rect class="rect" x="1158.5" y="596" width="13" height="13" fill="#e0e0e0" opacity=".8"/>
    <rect class="rect" x="1183.5" y="494.5" width="6" height="6" fill="#a0a0a0" opacity=".8"/>
    <rect class="rect" x="1190.5" y="501.5" width="6" height="6" fill="#e0e0e0" opacity=".8"/>
    <rect class="rect" x="1197.5" y="508.5" width="6" height="6" fill="#a0a0a0" opacity=".8"/>
    <rect class="rect" x="1204.5" y="515.5" width="6" height="6" fill="#e0e0e0" opacity=".8"/>
    <rect class="rect" x="1211.5" y="522.5" width="6" height="6" fill="#a0a0a0" opacity=".8"/>
    <rect class="rect" x="1218.5" y="529.5" width="6" height="6" fill="#e0e0e0" opacity=".8"/>
    <rect class="rect" x="1225.5" y="536.5" width="6" height="6" fill="#a0a0a0" opacity=".8"/>
    <rect class="rect" x="1232.5" y="543.5" width="6" height="6" fill="#e0e0e0" opacity=".8"/>
    <rect class="rect" x="1239.5" y="550.5" width="6" height="6" fill="#a0a0a0" opacity=".8"/>
    <rect class="rect" x="1246.5" y="557.5" width="6" height="6" fill="#e0e0e0" opacity=".8"/>
    <rect class="rect" x="1253.5" y="564.5" width="6" height="6" fill="#a0a0a0" opacity=".8"/>
    <rect class="rect" x="1260.5" y="571.5" width="6" height="6" fill="#e0e0e0" opacity=".8"/>
    <rect class="rect" x="1267.5" y="578.5" width="6" height="6" fill="#a0a0a0" opacity=".8"/>
    <rect class="rect" x="1274.5" y="585.5" width="6" height="6" fill="#e0e0e0" opacity=".8"/>
    <rect class="rect" x="1281.5" y="592.5" width="6" height="6" fill="#a0a0a0" opacity=".8"/>
    <rect class="rect" x="1288.5" y="599.5" width="6" height="6" fill="#e0e0e0" opacity=".8"/>
   </g>
   <g fill-opacity="0" stroke="#000" stroke-opacity=".8">
    <rect class="conv" x="842.07" y="555.32" width="3" height="3"/>
    <rect class="conv" x="950.65" y="564.98" width="3" height="3"/>
    <rect class="conv" x="1030.7" y="572.79" width="2" height="2"/>
    <rect class="conv" x="1159.7" y="599.42" width="3" height="3"/>
   </g>
   <g stroke="#000">
    <g stroke-opacity=".8" stroke-width=".5">
     <line class="link" x1="845.07" x2="963.95" y1="558.32" y2="574.31"/>
     <line class="link" x1="845.07" x2="963.95" y1="555.32" y2="574.31"/>
     <line class="link" x1="953.65" x2="1033.8" y1="567.98" y2="570.04"/>
     <line class="link" x1="953.65" x2="1033.8" y1="564.98" y2="570.04"/>
     <line class="link" x1="1032.7" x2="1162.7" y1="574.79" y2="601.79"/>
     <line class="link" x1="1032.7" x2="1162.7" y1="572.79" y2="601.79"/>
     <line class="link" x1="1162.7" x2="1289.2" y1="602.42" y2="601.55"/>
     <line class="link" x1="1162.7" x2="1289.2" y1="599.42" y2="601.55"/>
    </g>
    <polygon class="poly" points="1380.5 582 1370.5 582 1306.5 518 1316.5 518" fill="#e0e0e0" opacity=".8"/>
    <polygon class="poly" points="1402.5 555 1392.5 555 1382.5 545 1392.5 545" fill="#e0e0e0" opacity=".8"/>
    <g stroke-opacity=".8" stroke-width=".5">
     <line class="line" x1="1294.5" x2="1370.5" y1="605.5" y2="582"/>
     <line class="line" x1="1189.5" x2="1306.5" y1="494.5" y2="518"/>
     <line class="line" x1="1380.5" x2="1392.5" y1="582" y2="555"/>
     <line class="line" x1="1316.5" x2="1382.5" y1="518" y2="545"/>
    </g>
   </g>
   <g font-family="sans-serif" font-size="16px">
    <text class="text" x="873.25" y="645" dy="0.34999999em">Conv</text>
    <text class="text" x="973.5" y="645" dy="0.34999999em">MaxPool</text>
    <text class="text" x="1062" y="645" dy="0.34999999em">Conv</text>
    <text class="text" x="1190.25" y="645" dy="0.34999999em">MaxPool</text>
    <text class="text" x="1301.5" y="645" dy="0.34999999em">Flatten</text>
    <text class="info" x="802.5" y="512" dy="-0.30000001em">3@32x32</text>
    <text class="info" x="888.5" y="495.5" dy="-0.30000001em">8@30x30</text>
    <text class="info" x="977.5" y="503" dy="-0.30000001em">8@15x15</text>
    <text class="info" x="1053.5" y="476" dy="-0.30000001em">16@13x13</text>
    <text class="info" x="1183.5" y="479.5" dy="-0.30000001em">16@6x6</text>
    <text class="info" x="1306.5" y="503" dy="-0.30000001em">1x64</text>
    <text class="info" x="1382.5" y="530" dy="-0.30000001em">1x10</text>
   </g>
  </g>
 </g>
</svg>


# Training the model

Having prepared our data for training and defined our model, we can now start training our first model for this task!

We will start by instantiating our training and validation dataloaders, with the training dataloader configured to shuffle the training data. We will then instantiate our loss and optimiser before finally starting to train our model.

**Note**: earlier, we shifted our model to the GPU with `.to(device)` (if one is available), so we also need to shift our batches of data there too (also with `.to(device)`) to be on the same device as the model.


In [ ]:
BATCH_SIZE = 32

training_dataloader = torch.utils.data.DataLoader(
    training_dataset, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True
)

validation_dataloader = torch.utils.data.DataLoader(
    validation_dataset, batch_size=BATCH_SIZE
)

For this training task, we will use the [`CrossEntropyLoss`](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html) and the [`Adam`](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html) optimiser. Other optimisers are available, such as [`SGD`](https://pytorch.org/docs/stable/generated/torch.optim.SGD.html) (stochastic gradient descent, optionally with momentum).


In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimiser = optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
training_losses = []
validation_losses = []

In [ ]:
EPOCHS = 10

for epoch in range(EPOCHS):
    # Train the model for one epoch
    model.train()

    count = 0
    running_loss = 0.0

    for i, (inputs, labels) in enumerate(training_dataloader, 0):
        # Zero the parameter gradients
        optimiser.zero_grad()

        # Make predictions for the batch
        test_predictions = model(inputs.to(device))

        # Perform the backwards pass
        loss = criterion(test_predictions, labels.to(device))
        loss.backward()

        # Optimise the parameters
        optimiser.step()

        # Update our statistics
        size = test_predictions.size(0)
        running_loss += loss.item() * size
        count += size

        if i % 100 == 99:
            print(
                f"Training epoch: {epoch + 1:2d},  batch: {i + 1:5d},  loss: {running_loss / count:.3f}"
            )

    training_losses.append(running_loss / count)

    # Evaluate the model on the validation set
    model.eval()

    count = 0
    running_loss = 0.0

    with torch.inference_mode():
        for i, (inputs, labels) in enumerate(validation_dataloader, 0):
            # Make predictions for the batch
            test_predictions = model(inputs.to(device))

            # Compute the loss
            loss = criterion(test_predictions, labels.to(device))

            # Update our statistics
            size = test_predictions.size(0)
            running_loss += loss.item() * size
            count += size

        validation_losses.append(running_loss / count)
        print(f"Validation epoch: {epoch + 1:2d},  loss: {running_loss / count:.3f}")

Great &ndash; our model has finished training! 🥳

We can now take a look at our training and validation loss curves:


In [ ]:
plt.plot(training_losses, label="training")
plt.plot(validation_losses, label="validation")
plt.legend()

In addition to looking at the cross-entropy loss, we can also compute a final set of statistics (such as the model's accuracy) and a confusion matrix for the validation dataset:


In [ ]:
validation_predictions = []

model.eval()
with torch.inference_mode():
    for i, (inputs, labels) in enumerate(validation_dataloader, 0):
        # Select the class with the highest value for each instance in the dataset
        _, predicted_classes = model(inputs.to(device)).max(1)

        validation_predictions.extend(predicted_classes.cpu().numpy())

In [ ]:
from sklearn.metrics import classification_report, ConfusionMatrixDisplay

print(
    classification_report(
        y_true=y_valid, y_pred=validation_predictions, digits=3, target_names=CLASSES
    )
)

ConfusionMatrixDisplay.from_predictions(
    y_true=y_valid, y_pred=validation_predictions, display_labels=CLASSES
)

From this, we can gain a greater insight into the performance of our model, which may give us ideas as to how we could improve.


## Submitting our predictions to the DOXA AI platform

We are now ready to generate predictions for the test set and upload our work to the DOXA AI platform for evaluation! 👀

**Make sure to [enrol to take part](https://doxaai.com/competition/cifar-10) in the challenge if you have not already done so.**


In [ ]:
test_dataset = CIFAR10(features=data["test_data"], transform=test_transform)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [ ]:
test_predictions = []

model.eval()
with torch.inference_mode():
    for i, inputs in enumerate(test_dataloader, 0):
        # Select the class with the highest value for each instance in the test dataset
        _, predicted_classes = model(inputs.to(device)).max(1)

        test_predictions.extend(predicted_classes.cpu().numpy())

print(*test_predictions)

In [ ]:
# Prepare our submission
os.makedirs("submission", exist_ok=True)

with open("submission/doxa.yaml", "w") as f:
    f.write(
        "competition: cifar-10\nenvironment: cpu\nlanguage: python\nentrypoint: run.py\n"
    )

with open("submission/run.py", "w") as f:
    contents = "\\n".join([str(prediction) for prediction in test_predictions])
    f.write(
        f"""import os
with open(os.environ["DOXA_STREAMS"] + "/out", "w") as f:
    f.write("{contents}")"""
    )

Next, we need to make sure we are logged in:


In [ ]:
!doxa login

Finally, we can submit the predictions for evaluation:


In [ ]:
!doxa upload submission

Wooo! 🥳 You have just submitted your predictions to the platform &ndash; well done! Take a moment to see how well you have done on the [scoreboard](https://doxaai.com/competition/cifar-10/scoreboard).


## Next steps

Congratulations &ndash; you have made it to the end of this Jupyter notebook, and you will hopefully have just trained your first CNN! 👏

Your challenge now is to improve the performance of the model. Here are some ideas:

1. **Transforming and augmenting the training dataset**:

   - What other transformations or augmentations might you want to apply to the training dataset? (e.g. scaling, rotating, cropping, resizing, flipping, padding, distorting, blurring, solarising, erasing, etc)
   - Would colour transformations help? (e.g. mapping to greyscale, introducing colour jitter, adjusting the brightness, etc)
   - How could you randomly apply certain transformations (potentially in a random order)? (see the [documentation](https://pytorch.org/vision/main/transforms.html#composition))
   - Would adding random noise help?

2. **Improving the architecture of the model**:

   - How could you improve the architecture of this simple model? You may wish to look at the classic [LeNet](https://en.wikipedia.org/wiki/LeNet), [AlexNet](https://en.wikipedia.org/wiki/AlexNet) and VGG-16 models for inspiration.
   - Could [batch normalisation](https://en.wikipedia.org/wiki/Batch_normalization) help? What about different types of pooling?

3. **Improving the training setup**:
   - How could you improve the training process to improve performance? (both in terms of model accuracy and training time)

We hope that you have found this to be a useful and enjoyable exercise in exploring and gaining exposure to some fascinating ideas and concepts in machine learning. We look forward to seeing what you build! Do continue the conversation on the [DOXA Community Discord server](https://discord.gg/MUvbQ3UYcf). 😎
